In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [5]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artist
from plan import validate_plan

tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, search_tool, find_similar_artist]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Manual Test of Tool Infra

In [6]:
from langchain_core.messages import AIMessage, HumanMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_playlists",
            "args" : {},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

# tool_node.invoke({"messages": [message_with_single_tool_call]})

# Bind Tools to Model

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=0.8)
llm_with_tools = llm.bind_tools(tools)

# First Message

In [8]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
ai_tool_call_message = llm_with_tools.invoke([human_message])

2024-11-12 21:36:55,449 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
ai_tool_call_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_HR329a7K28WEMVH3hCxQpxxO)
 Call ID: call_HR329a7K28WEMVH3hCxQpxxO
  Args:
    plan: {'steps': [{'name': 'get_artists_from_playlist', 'description': "Retrieve the list of unique artists from the 'New Rock and Blues' playlist.", 'success_criteria': 'Successfully obtained a list of artists from the playlist along with their Spotify URIs.', 'tool': 'functions.get_artists_from_playlist', 'action': 'Tool Calling'}, {'name': 'find_similar_artists', 'description': "Find 3-4 similar artists for each artist from the 'New Rock and Blues' playlist.", 'success_criteria': 'For each artist in the original playlist, a list of 3-4 similar artists is obtained.', 'tool': 'functions.find_similar_artist', 'action': 'Tool Calling'}, {'name': 'filter_existing_artists', 'description': "Remove artists from the new artist list who are already present in the 'New Rock and Blues' playlist.", 'success

In [10]:
import json
plan = ai_tool_call_message.additional_kwargs["tool_calls"][0]["function"]["arguments"]
json.loads(plan)

{'plan': {'steps': [{'name': 'get_artists_from_playlist',
    'description': "Retrieve the list of unique artists from the 'New Rock and Blues' playlist.",
    'success_criteria': 'Successfully obtained a list of artists from the playlist along with their Spotify URIs.',
    'tool': 'functions.get_artists_from_playlist',
    'action': 'Tool Calling'},
   {'name': 'find_similar_artists',
    'description': "Find 3-4 similar artists for each artist from the 'New Rock and Blues' playlist.",
    'success_criteria': 'For each artist in the original playlist, a list of 3-4 similar artists is obtained.',
    'tool': 'functions.find_similar_artist',
    'action': 'Tool Calling'},
   {'name': 'filter_existing_artists',
    'description': "Remove artists from the new artist list who are already present in the 'New Rock and Blues' playlist.",
    'success_criteria': 'The list of new artists has no overlap with the original playlist artists.',
    'tool': 'functions.filter_artists',
    'action': 

# chat_prompt_template holds all messages (Human, AI, Tool)

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_tool_call_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, considering tempo, energy, and mood, using best 

# Check for Tools in Message

In [12]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

[StructuredTool(name='validate_plan', description='Validates a step-by-step plan to solve a problem\n\nArgs:\n    plan (Plan): a step-by-step plan to solve a problem\n\nReturns:\n    bool: whether plan is okay or not', args_schema=<class 'langchain_core.utils.pydantic.validate_plan'>, func=<function validate_plan at 0x000002980127E660>)]


# Tool Call (should be validate_plan())

In [13]:
response = tool_node.invoke({"messages": messages})
tool_message = response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: validate_plan

true


In [14]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

[HumanMessage(content="\n    Create and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, considering tempo, energy, and mood, using best 

# Send tool result to LLM

In [15]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

2024-11-12 21:36:56,994 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

### Step 1: Retrieve Artists from 'New Rock and Blues' Playlist

I'll start by retrieving the list of unique artists from the 'New Rock and Blues' playlist to identify which artists to find similar artists for. Let's proceed with this step.
Tool Calls:
  get_playlists (call_dFp22CaHwKicKGc0uoB3Vswo)
 Call ID: call_dFp22CaHwKicKGc0uoB3Vswo
  Args:


# Tool Call (should be get_playlists())

In [16]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})

In [17]:
playlist_tool_message: ToolMessage = response["messages"][0]
playlist_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_playlists

{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "descripti

In [18]:
# We need to check that is JSON serializable
# json.loads(playlist_tool_message.content)

# Format Message List for LLM

In [19]:
chat_prompt_template += playlist_tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [20]:
ai_tool_call_message = llm_with_tools.invoke(messages)
ai_tool_call_message.pretty_print()

2024-11-12 21:36:58,529 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_artists_from_playlist (call_jvDFLlZp06bijmKEQgVCLEtM)
 Call ID: call_jvDFLlZp06bijmKEQgVCLEtM
  Args:
    playlist_id: spotify:playlist:01tk0aitEuGK0ajWCkzdKc


In [21]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [ ]:
response = tool_node.invoke({"messages": messages})
track_list_tool_message: ToolMessage = response["messages"][0]

# Decode AI Tool Call message

In [ ]:
track_list_tool_message.pretty_print()

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool message containing playlist tracks to list of messages

In [ ]:
chat_prompt_template += track_list_tool_message
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
# llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='function_calling', include_raw=True)
# response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
# response
response = llm_with_tools.invoke(messages)

In [ ]:
response.pretty_print()

In [ ]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
print(tools_in_ai_message)
if len(tools_in_ai_message) == 0:
    raise ValueError

# Tool Call (find_similar_artitsts())

In [ ]:
response = tool_node.invoke({"messages": messages})

In [ ]:
check_artist_tool_message: ToolMessage = response["messages"][0]
check_artist_tool_message.pretty_print()

In [ ]:
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=check_artist_tool_message.content, name=check_artist_tool_message.name, tool_call_id=check_artist_tool_message.tool_call_id)
tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

In [ ]:
response = llm_with_tools.invoke(chat_prompt_template.format_messages())

In [ ]:
response.pretty_print()

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response